# Библиотеки и инициализация процессов

In [114]:
import pandas as pd
import praw
import snscrape.modules.twitter as sntwitter
import datetime
import requests
from tqdm.notebook import tqdm
import cryptocompare
import getpass
import math

api_key = getpass.getpass('Api Key for CryptoCompare parsing:')
client_id = getpass.getpass('Cliend id for Reddit parsing:')
client_secret = getpass.getpass('Cliend secret for Reddit parsing:')



from IPython.core.display import display, HTML
display(HTML('<style>.container {width: 100% !important; } </style>'))

Api Key for CryptoCompare parsing:········
Cliend id for Reddit parsing:········
Cliend secret for Reddit parsing:········


# Парсинг

## Reddit

author - автор поста - сообщество или персона

title - заголовок

score - количество лайков поста

id - id поста

subreddit - один из тэгов для парсинга (['Bitcoin','Crypto','Cryptocurrency','Cryptocurrencies','Blockchain','Binance'])

num_comments - общее число комментариев

body - содержание поста (только текст)

created - время создания

top 1-10 comment - содержание комментария

top 1-10 comment score - количество лайков комментария

top 1-10 comment author - автор комментария

top 1-10 comment time - время комментария

total comments score - сумма лайков комментариев под постом

subreddit subscribers - количетсво подписчиков (если пост оставлен пабликом)

In [ ]:
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent='ParsBars')

posts_total = []
subreddit_list = ['Bitcoin','Crypto','Cryptocurrency','Cryptocurrencies','Blockchain','Binance']

for x in subreddit_list:
    posts = []
    ml_subreddit = reddit.subreddit(x).hot(limit=1000000)
    sub_c = reddit.subreddit(x)
    sub_c = sub_c.subscribers
    
    for post in tqdm(ml_subreddit):
        posts_loc = []
        posts_loc.append([post.author,post.title, post.score, post.id, post.subreddit, post.num_comments, post.selftext, datetime.datetime.fromtimestamp(post.created)])
        submission = reddit.submission(id=post.id)
        comm = []
        scor = []
        aut = []
        tim = []
        submission.comments.replace_more(limit=0)
        for top_level_comment in submission.comments:
            comm += [top_level_comment.body]
            scor += [top_level_comment.score]
            aut += [top_level_comment.author]
            tim += [datetime.datetime.fromtimestamp(top_level_comment.created)]
        df = pd.DataFrame([comm,scor,aut,tim],index=['comment','comment_score','aut','tim']).T.sort_values(by='comment_score',ascending=False).iloc[:10]
        shap = min(df.shape[0],10)
        comm_df = pd.DataFrame(df.iloc[:,0].tolist()+df.iloc[:,1].tolist()+df.iloc[:,2].tolist() + df.iloc[:,3].tolist() \
                               + [sum(scor),sub_c],index= ['top {} comment'.format(x) for x in range(1,shap+1)]\
                               + ['top {} comment score'.format(x) for x in range(1,shap+1)] \
                               + ['top {} comment author'.format(x) for x in range(1,shap+1)] \
                               + ['top {} comment time'.format(x) for x in range(1,shap+1)] \
                               + ['total comments score','subreddit subscribers']).T
        posts +=[pd.concat([pd.DataFrame(posts_loc,columns=['author','title', 'score', 'id', 'subreddit', 'num_comments', 'body', 'created']),comm_df],axis=1)]
    
    posts = pd.concat(posts)
    posts_total += [posts]
posts_total = pd.concat(posts_total).to_csv('reddit.csv')

## Twitter

date - время создания

id - id поста

content - содержание поста (только текстовая составляющая)

username - имя автора поста

subreddit - один из тэгов для парсинга (['Bitcoin','Crypto','Cryptocurrency','Cryptocurrencies','Blockchain','Binance'])

In [ ]:
maxTweets = 1000000

total_tweet = []
subreddit_list = ['Bitcoin','Crypto','Cryptocurrency','Cryptocurrencies','Blockchain','Binance']
for x in subreddit_list:
    posts_total = []
    try:
        for i,tweet in tqdm(enumerate(sntwitter.TwitterSearchScraper(x).get_items())):
            if i>maxTweets:
                break
            posts_total.append({'date':tweet.date, 
                                 'id':tweet.id, 
                                 'content':tweet.content, 
                                 'username':tweet.username})
    except:
        pass
    
    posts_total = pd.DataFrame(posts_total)
    posts_total['subreddit'] = x
    total_tweet += [posts_total]
posts_total = pd.concat(total_tweet).to_csv('twitter.csv')

## Bitcoin price 

Минимальное время спарсенных твиттов/реддитов - '2017-12-14 15:38:42'

Максимальное время спарсенных твиттов/реддитов - '2021-09-24 07:03:18'

In [92]:
start_point = int(datetime.datetime.timestamp(datetime.datetime.strptime('2017-12-14 15:38:42', "%Y-%m-%d %H:%M:%S")))
end_point = int(datetime.datetime.timestamp(datetime.datetime.strptime('2021-09-24 07:03:18', "%Y-%m-%d %H:%M:%S")))
days_between = (datetime.datetime.strptime('2021-09-24 07:03:18', "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime('2017-12-14 15:38:42', "%Y-%m-%d %H:%M:%S")).days+1
hours_between = days_between*24
minutes_between = hours_between*60

### Daily

time - время среза данных

high - максимальная цена

low - минимальная цена

open - цена открытия

volumefrom - минимальный объем сделки за период

volumeto - максимальный объем сделки за период

close - цена закрытия 

conversionType - техническое поле

conversionSymbol - техническое поле

top_tier_volume_quote - максимальный sell-side объем по BTC

top_tier_volume_base - максимальный buy-side объем по BTC

top_tier_volume_total - максимальный суммарный объем по BTC

cccagg_volume_quote - средневзешенный sell-side объем по BTC

cccagg_volume_base - средневзешенный buy-side объем по BTC

cccagg_volume_total - средневзешенный суммарный объем по BTC

total_volume_quote - суммарный sell-side объем по BTC

total_volume_base - суммарный buy-price объем по BTC

total_volume_total - суммарный объем по BTC

volume - суммарный объем биржи по всем валютам

In [136]:
assets = 'BTC'

# В целом по биржам
url_daily_exchange_vol = 'https://min-api.cryptocompare.com/data/exchange/histoday?tsym=USD&limit={}'.format(days_between) + api_key
data_exchange = pd.DataFrame(requests.get(url_daily_exchange_vol).json()['Data']).iloc[:,1:]


url_daily_pair = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym={}&tsym=USD&limit={}'.format(assets,days_between) + api_key
# По активу
url_daily_symbol_vol = 'https://min-api.cryptocompare.com/data/symbol/histoday?fsym={}&tsym=USD&limit={}'.format(assets,days_between) + api_key

try:
    data_main = pd.DataFrame(requests.get(url_daily_pair).json()['Data']['Data'])
    data_vol = pd.DataFrame(requests.get(url_daily_symbol_vol).json()['Data']).iloc[:,1:]
    data = pd.concat([data_main,data_vol,data_exchange],axis=1)
except KeyError:
    print("I can't find "+assets + ' in USD')
data.to_csv('BTC day.csv')

In [159]:
data = pd.read_csv('BTC day.csv',index_col=0)
print(data.shape)
data.head()

(1381, 19)


,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,top_tier_volume_quote,top_tier_volume_base,top_tier_volume_total,cccagg_volume_quote,cccagg_volume_base,cccagg_volume_total,total_volume_quote,total_volume_base,total_volume_total,volume
0,1513209600,16941.08,16023.64,16286.82,107918.03,1.773814e+09,16467.91,direct,NaN,6.344126e+09,3.229618e+09,9.573744e+09,6.736640e+09,3.459583e+09,1.019622e+10,1.373610e+10,8.712356e+09,2.244846e+10,1.631186e+10
1,1513296000,17987.03,16442.20,16467.91,153651.15,2.682351e+09,17604.85,direct,NaN,5.399470e+09,4.468816e+09,9.868286e+09,6.355843e+09,5.091872e+09,1.144772e+10,1.241196e+10,1.254010e+10,2.495207e+10,1.631425e+10
2,1513382400,19587.70,17318.54,17594.08,112173.97,2.078806e+09,19345.49,direct,NaN,5.083908e+09,3.839717e+09,8.923624e+09,5.958368e+09,4.369942e+09,1.032831e+10,1.164882e+10,1.164733e+10,2.329614e+10,1.392422e+10
3,1513468800,19870.62,18750.91,19346.60,117408.38,2.264650e+09,19065.71,direct,NaN,5.667905e+09,4.075019e+09,9.742924e+09,6.602854e+09,4.676746e+09,1.127960e+10,1.295949e+10,1.208859e+10,2.504809e+10,1.526174e+10
4,1513555200,19221.10,18114.42,19065.71,139251.39,2.597510e+09,18972.32,direct,NaN,6.221768e+09,4.097386e+09,1.031915e+10,7.216912e+09,4.821051e+09,1.203796e+10,1.413224e+10,1.168359e+10,2.581583e+10,1.781935e+10


### Hourly

time - время среза данных

high - максимальная цена

low - минимальная цена

open - цена открытия

volumefrom - минимальный объем сделки за период

volumeto - максимальный объем сделки за период

close - цена закрытия 

conversionType - техническое поле

conversionSymbol - техническое поле

top_tier_volume_quote - максимальный sell-side объем по BTC

top_tier_volume_base - максимальный buy-side объем по BTC

top_tier_volume_total - максимальный суммарный объем по BTC

cccagg_volume_quote - средневзешенный sell-side объем по BTC

cccagg_volume_base - средневзешенный buy-side объем по BTC

cccagg_volume_total - средневзешенный суммарный объем по BTC

total_volume_quote - суммарный sell-side объем по BTC

total_volume_base - суммарный buy-side объем по BTC

total_volume_total - суммарный объем по BTC

volume - суммарный объем биржи по всем валютам

In [163]:
points_of_time = [int(datetime.datetime.timestamp((datetime.datetime.fromtimestamp(end_point)-datetime.timedelta(hours=x*2000))))  for x in range(math.ceil(hours_between/2000)+1)]
data_part = []
for p in points_of_time:
    # В целом по биржам
    url_hourly_exchange_vol = 'https://min-api.cryptocompare.com/data/exchange/histohour?tsym=USD&limit=2000&toTs={}'.format(p) + api_key
    data_exchange = pd.DataFrame(requests.get(url_hourly_exchange_vol).json()['Data']).iloc[:,1:]
    
    
    url_hourly_pair = 'https://min-api.cryptocompare.com/data/v2/histohour?fsym={}&tsym=USD&limit=2000&toTs={}'.format(assets,p) + api_key
    # По активу
    url_hourly_symbol_vol = 'https://min-api.cryptocompare.com/data/symbol/histohour?fsym={}&tsym=USD&limit=2000&toTs={}'.format(assets,p) + api_key
    
    try:
        data_main = pd.DataFrame(requests.get(url_hourly_pair).json()['Data']['Data'])
        data_vol = pd.DataFrame(requests.get(url_hourly_symbol_vol).json()['Data']).iloc[:,1:]
        data = pd.concat([data_main,data_vol,data_exchange],axis=1)
        data_part += [data]
    except KeyError:
        print("I can't find "+assets + ' in USD')
pd.concat(data_part).sort_values(by='time').reset_index(drop=True).to_csv('BTC hour.csv')

In [164]:
data = pd.read_csv('BTC hour.csv',index_col=0)
print(data.shape)
data.head()

(36018, 19)


,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,top_tier_volume_quote,top_tier_volume_base,top_tier_volume_total,cccagg_volume_quote,cccagg_volume_base,cccagg_volume_total,total_volume_quote,total_volume_base,total_volume_total,volume
0,1502856000,4080.96,4032.19,4080.03,3266.45,13249764.64,4053.84,direct,NaN,25218452.18,30648003.32,5.586646e+07,28807478.88,36810407.66,6.561785e+07,57442953.34,9.104171e+07,1.484847e+08,9.903669e+07
1,1502859600,4053.96,3968.42,4053.84,5907.95,23589732.03,3993.01,direct,NaN,30942393.51,64357618.19,9.530001e+07,34289933.44,73839180.15,1.081291e+08,68506235.79,1.810578e+08,2.495640e+08,1.558119e+08
2,1502863200,4069.19,3938.62,3993.01,5992.99,23936815.84,4059.10,direct,NaN,32662359.97,73799633.65,1.064620e+08,36420477.71,84417386.84,1.208379e+08,72747920.84,1.967476e+08,2.694955e+08,1.662107e+08
3,1502866800,4116.13,4059.10,4059.10,5135.04,21014644.00,4073.09,direct,NaN,30216462.55,60406572.21,9.062303e+07,34485101.60,77668694.21,1.121538e+08,67979586.98,1.844852e+08,2.524647e+08,1.591073e+08
4,1502870400,4166.65,4059.12,4073.09,4839.76,19930237.00,4163.84,direct,NaN,27455902.94,50130343.49,7.758625e+07,31408927.72,63922896.93,9.533187e+07,62542375.78,1.533012e+08,2.158436e+08,1.388617e+08


### Minutely

Unix - время в формате Timestamp 

Date - обычный datetime формат

Symbol - BTC/USD - техническое поле

Open - Цена открытия

High - Максимальная цена

Low - Минимальная цена

Close - Цена закрытия

Volume BTC - volume_quote - sell-side объем по BTC

Volume USD - volume_base - суммарный buy-side объем по BTC

In [165]:
pd.concat([pd.read_csv('https://www.cryptodatadownload.com/cdd/Bitstamp_BTCUSD_{}_minute.csv'.format(x),skiprows=1) for x in [2017,2018,2019,2020,2021]]).to_csv('BTC minute.csv')

In [166]:
data = pd.read_csv('BTC minute.csv',index_col=0)
print(data.shape)
data.head()

C:\Users\User\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(2447996, 9)


,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1514764740,2017-12-31 23:59:00,BTC/USD,13913.28,13913.28,13867.18,13880.00,0.591748,8213.456549
1,1514764680,2017-12-31 23:58:00,BTC/USD,13913.26,13953.83,13884.69,13953.77,1.398784,19518.309658
2,1514764620,2017-12-31 23:57:00,BTC/USD,13908.73,13913.26,13874.99,13913.26,0.775012,10782.944294
3,1514764560,2017-12-31 23:56:00,BTC/USD,13827.00,13908.69,13827.00,13859.58,0.666459,9236.841134
4,1514764500,2017-12-31 23:55:00,BTC/USD,13825.05,13825.05,13825.05,13825.05,0.065501,905.560130
